In [15]:
#making final schedules
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('traintripper_trainschedules.csv')

durations = []
for train_number in df['train_number'].unique():
    relevant_data = df[df['train_number'] == train_number]
    duration = 0
    prev_day_count = relevant_data.iloc[0]['day_count']

    for i in range(len(relevant_data)):
        if i==0:
            duration = 0
            durations.append(duration)
            departure = datetime.strptime(str(relevant_data.iloc[i]['departure_time']), '%H:%M:%S')
            continue
        else:
            arrival = datetime.strptime(str(relevant_data.iloc[i]['arrival_time']), '%H:%M:%S')
            if relevant_data.iloc[i]['day_count'] > prev_day_count:
                day = int(relevant_data.iloc[i]['day_count']) - int(prev_day_count)
                arrival += timedelta(days=day) # Add a day to the arrival time
            diff = arrival - departure
            duration = diff.total_seconds()/60 

            durations.append(duration)
        

df['duration_minutes'] = durations


# to csv
df.to_csv('final_schedules.csv', index=False)
df

station_code arrival_time departure_time  route_number  distance  \
0               MJ          NaN       08:30:00             1         0   
1              FLD     09:20:00       09:35:00             1        26   
2              KBK     11:00:00       14:40:00             1        47   
3              FLD     16:00:00       16:15:00             1        69   
4             MJMG     17:20:00            NaN             1        94   
...            ...          ...            ...           ...       ...   
71051         KOTA     17:40:00       17:50:00             1       778   
71052          RTM     21:45:00       21:55:00             1      1045   
71053          BRC     02:55:00       03:00:00             1      1306   
71054           ST     05:00:00       05:03:00             1      1435   
71055           BL     06:30:00       06:31:00             1      1504   

       day_count  train_number  halt_time_minutes  duration_minutes  
0              1           961                 -1               0.0  
1              1           961                900              50.0  
2              1           961              13200             150.0  
3              1           961                900             450.0  
4              1           961                 -1             530.0  
...          ...           ...                ...               ...  
71051          1          4125                600             740.0  
71052          1          4125                600             985.0  
71053          2          4125                300            1295.0  
71054          2          4125                180            1420.0  
71055          2          4125                 60            1510.0  

[71056 rows x 9 columns]

In [20]:
# one hat encoding
df2 = pd.read_csv('../irctc/static_coverage.csv')

dummies = pd.get_dummies(df2['classCode'])
merged = pd.concat([df2, dummies], axis='columns')
final = merged.drop(['classCode'], axis='columns')
final

baseFare  reservationCharge  superfastCharge  fuelAmount  \
0           1059                 60                0         0.0   
1            626                 50                0         0.0   
2            441                 40                0         0.0   
3            125                 20                0         0.0   
4           1059                 60                0         0.0   
...          ...                ...              ...         ...   
296050      1910                 40               45         0.0   
296051       755                 20               30         0.0   
296052      2790                 50               45         0.0   
296053      1910                 40               45         0.0   
296054       755                 20               30         0.0   

        totalConcession  tatkalFare  serviceTax  otherCharge  cateringCharge  \
0                     0           0        56.0            0               0   
1                     0           0        34.0            0               0   
2                     0           0        24.0            0               0   
3                     0           0         0.0            0               0   
4                     0           0        56.0            0               0   
...                 ...         ...         ...          ...             ...   
296050                0           0       100.0            0               0   
296051                0           0         0.0            0               0   
296052                0           0       145.0            0               0   
296053                0           0       100.0            0               0   
296054                0           0         0.0            0               0   

        dynamicFare  ...  trainNumber                   timeStamp  \
0                 0  ...        11464  2023-10-03 22:13:07.781307   
1                 0  ...        11464  2023-10-03 22:13:07.781307   
2                 0  ...        11464  2023-10-03 22:13:07.781307   
3                 0  ...        11464  2023-10-03 22:13:07.781307   
4                 0  ...        11464  2023-10-03 22:13:07.781307   
...             ...  ...          ...                         ...   
296050            0  ...        22645  2023-10-30 07:06:00.395678   
296051            0  ...        22645  2023-10-30 07:06:00.395678   
296052            0  ...        22645  2023-10-30 07:06:00.395678   
296053            0  ...        22645  2023-10-30 07:06:00.395678   
296054            0  ...        22645  2023-10-30 07:06:00.395678   

        fromStnCode toStnCode 1A 2A  2S  3A  CC  SL  
0               JBP      SRID  1  0   0   0   0   0  
1               JBP      SRID  0  1   0   0   0   0  
2               JBP      SRID  0  0   0   1   0   0  
3               JBP      SRID  0  0   0   0   0   1  
4               JBP       KKB  1  0   0   0   0   0  
...             ...       ... .. ..  ..  ..  ..  ..  
296050           ET       ERN  0  0   0   1   0   0  
296051           ET       ERN  0  0   0   0   0   1  
296052           ET       ERS  0  1   0   0   0   0  
296053           ET       ERS  0  0   0   1   0   0  
296054           ET       ERS  0  0   0   0   0   1  

[296055 rows x 22 columns]

In [21]:
final.to_csv("static_coverage_hot_encoded.csv", index=False)

In [25]:
df_schedules = df
df_static = final

In [31]:
import pandas as pd

df = pd.read_csv('final_schedules.csv')

distance_map = {}

for index, row in df.iterrows():
    key = (row['train_number'], row['station_code'] )
    distance = row['distance']
    duration = row['duration_minutes']
    # distance_map[key] = distance
    distance_map[key] = (distance, duration)

static_df = pd.read_csv('static_coverage_hot_encoded.csv')

# Create a new 'distance' column and initialize it with None
static_df['distance'] = None
static_df['duration'] = None

# Iterate through the rows of the static DataFrame and populate the 'distance' column
for index, row in static_df.iterrows():
    train_number = row['trainNumber']
    from_stn = row['fromStnCode']
    to_stn = row['toStnCode']
    
    # Look up the distance in the distance_map using trainNumber and fromStn
    key1 = distance_map.get((train_number, from_stn), None)
    key2 = distance_map.get((train_number, to_stn), None)
    
    cal_distance = key2[0] - key1[0]
    cal_duration = key2[1] - key1[1]
    
    
    # Update the 'distance' column with the distance value
    static_df.at[index, 'distance'] = cal_distance
    static_df.at[index, 'duration'] = cal_duration
    static_df
    

In [32]:
static_df

baseFare  reservationCharge  superfastCharge  fuelAmount  \
0           1059                 60                0         0.0   
1            626                 50                0         0.0   
2            441                 40                0         0.0   
3            125                 20                0         0.0   
4           1059                 60                0         0.0   
...          ...                ...              ...         ...   
296050      1910                 40               45         0.0   
296051       755                 20               30         0.0   
296052      2790                 50               45         0.0   
296053      1910                 40               45         0.0   
296054       755                 20               30         0.0   

        totalConcession  tatkalFare  serviceTax  otherCharge  cateringCharge  \
0                     0           0        56.0            0               0   
1                     0           0        34.0            0               0   
2                     0           0        24.0            0               0   
3                     0           0         0.0            0               0   
4                     0           0        56.0            0               0   
...                 ...         ...         ...          ...             ...   
296050                0           0       100.0            0               0   
296051                0           0         0.0            0               0   
296052                0           0       145.0            0               0   
296053                0           0       100.0            0               0   
296054                0           0         0.0            0               0   

        dynamicFare  ...  fromStnCode toStnCode  1A 2A 2S 3A  CC  SL  \
0                 0  ...          JBP      SRID   1  0  0  0   0   0   
1                 0  ...          JBP      SRID   0  1  0  0   0   0   
2                 0  ...          JBP      SRID   0  0  0  1   0   0   
3                 0  ...          JBP      SRID   0  0  0  0   0   1   
4                 0  ...          JBP       KKB   1  0  0  0   0   0   
...             ...  ...          ...       ...  .. .. .. ..  ..  ..   
296050            0  ...           ET       ERN   0  0  0  1   0   0   
296051            0  ...           ET       ERN   0  0  0  0   0   1   
296052            0  ...           ET       ERS   0  1  0  0   0   0   
296053            0  ...           ET       ERS   0  0  0  1   0   0   
296054            0  ...           ET       ERS   0  0  0  0   0   1   

        distance  duration  
0             54      33.0  
1             54      33.0  
2             54      33.0  
3             54      33.0  
4             69      49.0  
...          ...       ...  
296050      2087    2132.0  
296051      2087    2132.0  
296052      2090    2145.0  
296053      2090    2145.0  
296054      2090    2145.0  

[296055 rows x 24 columns]

In [33]:
static_df.to_csv('final_for_model.csv', index=False)